<a href="https://colab.research.google.com/github/CryptoRobotFr/General-code-trading-bot/blob/main/order_book_price_impact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt

In [ ]:
import ccxt
import pandas as pd

def get_spread_from_order_book(exchange, symbol, usd_amount, show_order=False):
    df_bid = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['bids'], columns=['price', 'quantity'])
    df_ask = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['asks'], columns=['price', 'quantity'])
    df_bid['usd_quantity'] = df_bid['price'] *  df_bid['quantity']
    df_ask['usd_quantity'] = df_ask['price'] *  df_ask['quantity']

    i = 0
 
    for j in range(0,2,1):
        try: 
            operations_array = []
            usd_spent = 0
            if j == 0:
                df = df_ask.copy()
                # print(df)
            else:
                df = df_bid.copy()  
                # print(df)
            while(usd_spent < usd_amount):
                if df.iloc[i]['usd_quantity'] > usd_amount - usd_spent:
                    operations_array.append({"amount":usd_amount - usd_spent, "price":df.iloc[i]['price']})
                    usd_spent += (usd_amount - usd_spent)
                else: 
                    operations_array.append({"amount":df.iloc[i]['usd_quantity'], "price":df.iloc[i]['price']})
                    usd_spent += df.iloc[i]['usd_quantity']
                i += 1
                
            dt = pd.DataFrame(operations_array)
            dt['sum'] = dt['amount'] * dt['price']
            mean_price = dt['sum'].sum() / dt['amount'].sum()
            mid_bid_ask_price = (df_bid.iloc[0]["price"]+df_ask.iloc[0]["price"])/2
            diff_spread = abs(mid_bid_ask_price - mean_price) / mean_price
            spread_fee = usd_amount * diff_spread
            del dt['sum']
            if j == 0:
                print("||| Mid Bid Ask Price :",mid_bid_ask_price,"$ |||")
                print("\n------------ BUY ------------")
            else:
                print("\n------------ SELL ------------")
            
            print("||| Mean Operation Price  =",mean_price,"$ |||")
            # print("||| Mid Bid Ask Price =", mid_bid_ask_price,"$ |||")
            print("||| Spread Differance =", round(diff_spread*100,4),"% |||")
            print("||| Spread Fees =", round(spread_fee,2),"$ |||")
            if show_order:
                print("-"*30)
                print("--- Operation array ---")
                print(dt.to_string(index=False))
        except:
            print("Error, you have used all orders possible in the",len(df),"first orders of orderbook | Last one was at",df.iloc[-1]['price'],"$")




In [ ]:
exchange = ccxt.ftx()
get_spread_from_order_book(
    exchange = exchange,
    symbol = "MKR/USD", 
    usd_amount = 10000,
    show_order = False
)

||| Mid Bid Ask Price : 2218.0 $ |||

------------ BUY ------------
||| Mean Operation Price  = 2220.7020261750004 $ |||
||| Spread Differance = 0.1217 % |||
||| Spread Fees = 12.17 $ |||

------------ SELL ------------
||| Mean Operation Price  = 2213.29427555 $ |||
||| Spread Differance = 0.2126 % |||
||| Spread Fees = 21.26 $ |||


In [ ]:
exchange = ccxt.ftx()
get_spread_from_order_book(
    exchange = exchange,
    symbol = "ETH/USDT", 
    usd_amount = 500000,
    show_order = False
)

||| Mid Bid Ask Price : 3193.3500000000004 $ |||

------------ BUY ------------
||| Mean Operation Price  = 3195.2467621259993 $ |||
||| Spread Differance = 0.0594 % |||
||| Spread Fees = 296.81 $ |||

------------ SELL ------------
||| Mean Operation Price  = 3182.89420074436 $ |||
||| Spread Differance = 0.3285 % |||
||| Spread Fees = 1642.5 $ |||
